nn.DataParallel()只能处理模型

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 定义神经网络模型
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # 定义你的模型结构
        self.fc = nn.Linear(in_features=10, out_features=1)
    
    def forward(self, x):
        # 定义前向传播逻辑
        x = self.fc(x)
        return x

# 定义数据集
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

# 假设有10张图片数据
data = torch.randn(10, 10)  # 假设每张图片有10个特征

# 创建数据集和数据加载器
dataset = MyDataset(data)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# 初始化模型并放到GPU上
model = MyModel().cuda()

# 将模型在GPU上进行并行化
model = nn.DataParallel(model)


In [12]:
x = 0
for batch in dataloader:
  # 将数据移动到GPU上
  inputs = batch.cuda()
  
  # 正向传播
  outputs = model(inputs)
  x = x+1
  print(x)
  print(outputs)

1
tensor([[0.3122]], device='cuda:0', grad_fn=<AddmmBackward0>)
2
tensor([[0.2484]], device='cuda:0', grad_fn=<AddmmBackward0>)
3
tensor([[0.3374]], device='cuda:0', grad_fn=<AddmmBackward0>)
4
tensor([[-0.1192]], device='cuda:0', grad_fn=<AddmmBackward0>)
5
tensor([[0.5996]], device='cuda:0', grad_fn=<AddmmBackward0>)
6
tensor([[0.2356]], device='cuda:0', grad_fn=<AddmmBackward0>)
7
tensor([[-0.1340]], device='cuda:0', grad_fn=<AddmmBackward0>)
8
tensor([[0.0684]], device='cuda:0', grad_fn=<AddmmBackward0>)
9
tensor([[-1.0539]], device='cuda:0', grad_fn=<AddmmBackward0>)
10
tensor([[-0.1103]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

num_epochs = 1000

# 训练模型
for epoch in range(num_epochs):
    for batch in dataloader:
        # 将数据移动到GPU上
        inputs = batch.cuda()
        
        # 正向传播
        outputs = model(inputs)
        
        # 计算损失
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


torch.multiprocessing，可以给函数使用

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp

# 定义神经网络模型
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # 定义你的模型结构
        self.fc = nn.Linear(in_features=1, out_features=1)
    
    def forward(self, x):
        # 定义前向传播逻辑
        x = self.fc(x)
        return x

# 定义一个函数，用于在多进程中执行神经网络的计算
def worker_function(index, input_data):
    # 初始化模型
    model = MyModel()
    
    # 定义损失函数和优化器
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    # 准备数据
    inputs = torch.tensor([[input_data[index]]], dtype=torch.float32)
    labels = torch.tensor([[input_data[index] * 2]], dtype=torch.float32)  # 举例：输出为输入的两倍
    
    # 训练模型
    for _ in range(100):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # 返回输出结果
    return outputs.item()

if __name__ == '__main__':
    # 准备数据
    input_data = [1, 2, 3, 4, 5]
    
    # 创建进程池
    num_processes = len(input_data)
    pool = mp.Pool(processes=num_processes)
    
    # 在进程池中并行执行神经网络计算
    results = [pool.apply_async(worker_function, args=(i, input_data)) for i in range(num_processes)]
    
    # 收集所有进程的结果
    final_results = [result.get() for result in results]
    
    # 输出结果
    print("Final Results:", final_results)


Process 3: Result = tensor([ 3,  6,  9, 12, 15])Process 0: Result = tensor([0, 0, 0, 0, 0])Process 2: Result = tensor([ 2,  4,  6,  8, 10])Process 1: Result = tensor([1, 2, 3, 4, 5])





In [39]:
import torch
import numpy as np

# 假设你有一个输入张量 input_tensor
# input_tensor 的形状为 [batch_size, history_idx, cam_id, channel, width, height]

# 生成示例输入数据（假设batch_size=2，history_idx=3，cam_id=4，channel=3，width=5，height=5）
batch_size = 2
history_idx = 3
cam_id = 4
channel = 3
width = 5
height = 5

input_tensor = torch.randn(batch_size, history_idx, cam_id, channel, width, height)

print(f"{input_tensor.shape=}")

# 将输入张量重塑成 [batch_size*history_idx, cam_id, channel, width, height]
input_shape = input_tensor.shape
target_shape = np.append(-1, input_shape[2:])
print(f"{target_shape}")
print([-1, cam_id, channel, width, height])

reshaped_tensor = input_tensor.view(-1, cam_id, channel, width, height)
reshaped_tensor1 = input_tensor.view(target_shape[0], target_shape[1], target_shape[2], target_shape[3], target_shape[4])

# 恢复原来的形状
output_tensor = reshaped_tensor.view(batch_size, history_idx, cam_id, channel, width, height)

# 打印结果
print("原始形状：", input_tensor.shape)
print("重塑后的形状：", reshaped_tensor.shape)
print(input_tensor[1,2,0,0,0,2])
print(reshaped_tensor[5,0,0,0,2])
print(output_tensor[1,2,0,0,0,2])
print("重塑后的形状：", reshaped_tensor1.shape)

print("恢复后的形状：", output_tensor.shape)


input_tensor.shape=torch.Size([2, 3, 4, 3, 5, 5])
[-1  4  3  5  5]
[-1, 4, 3, 5, 5]
原始形状： torch.Size([2, 3, 4, 3, 5, 5])
重塑后的形状： torch.Size([6, 4, 3, 5, 5])
tensor(0.1753)
tensor(0.1753)
tensor(0.1753)
重塑后的形状： torch.Size([6, 4, 3, 5, 5])
恢复后的形状： torch.Size([2, 3, 4, 3, 5, 5])


In [48]:
import numpy as np
import cv2 as cv

image = np.ones([480,640,3])

image_dict = dict()
image_dict["wrist"] = image

image_list = []
for idx in range(5):
  image_list.append(image_dict)

image2 = np.ones([120,160,3])
image_dict2 = dict()
image_dict2["wrist"] = image2
image_list.append(image_dict2)

print("图片列表长度：",len(image_list)) # 妈的大问题，如果一个list又形状不同的元素，就无法使用np.shape

# 查看列表shape
# print("查看列表shape:")
# for idx in range(len(image_list)):
#   print(np.shape(image_list[idx]["wrist"]))


# for idx in range(2):
#   image_list[idx]["wrist"] = cv.resize(image_list[idx]["wrist"], (0, 0), fx=0.25, fy=0.25, interpolation=cv.INTER_LINEAR)

for idx in range(2):
  image_list[idx]["wrist"] = cv.resize(image_list[idx]["wrist"], (0, 0), fx=0.25, fy=0.25, interpolation=cv.INTER_LINEAR)

# 查看列表shape
print("查看列表shape:")
for idx in range(len(image_list)):
  print(np.shape(image_list[idx]["wrist"]))
  

图片列表长度： 6
查看列表shape:
(30, 40, 3)
(30, 40, 3)
(30, 40, 3)
(30, 40, 3)
(30, 40, 3)
(120, 160, 3)


In [49]:
import numpy as np
import cv2 as cv

image = np.ones([480,640,3])

image_dict = dict() # 又是dict的对象指针问题，操你妈
image_dict["wrist"] = image

image_list = []
for idx in range(5):
  image_list.append(image_dict.copy())  # 使用 .copy() 方法来避免列表中所有元素指向同一字典

image2 = np.ones([120,160,3])
image_dict2 = dict()
image_dict2["wrist"] = image2
image_list.append(image_dict2)

print(len(image_list))

# 统一图像大小
# for idx in range(len(image_list)):
#     image_list[idx]["wrist"] = cv.resize(image_list[idx]["wrist"], (640, 480), interpolation=cv.INTER_LINEAR)

for idx in range(2):
  image_list[idx]["wrist"] = cv.resize(image_list[idx]["wrist"], (0, 0), fx=0.25, fy=0.25, interpolation=cv.INTER_LINEAR)

for idx in range(len(image_list)):
  print(np.shape(image_list[idx]["wrist"]))


6
(120, 160, 3)
(120, 160, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(120, 160, 3)


In [7]:
import numpy as np

np.random.choice(range(15,20), size=3, replace=False)

array([18, 15, 17])

In [19]:
x = [10, 5]
# x = reversed(x)

# print(x[-1])

y = 12

for i in x:
  
  if y >= i:
    print(i)
    break
  
  elif y < i and i != x[-1]:
    continue
  
  print(i)

10


In [23]:
for i in range(10, 0, -1):
  print(i)

10
9
8
7
6
5
4
3
2
1


In [16]:
import torch

# 创建一个三维张量
tensor = torch.tensor([[[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]],
                       [[10, 11, 12],
                        [13, 14, 15],
                        [16, 17, 18]]]).float()

# 计算每个“列”的平均值
column_means = torch.mean(tensor, dim=1)

# 创建一个新“行”并将平均值添加进去
new_row = torch.unsqueeze(column_means, dim=1)

# 将新“行”与原始张量在第一维上拼接
extended_tensor = torch.cat((tensor, new_row), dim=1)

print(extended_tensor)


tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [ 7.,  8.,  9.],
         [ 4.,  5.,  6.]],

        [[10., 11., 12.],
         [13., 14., 15.],
         [16., 17., 18.],
         [13., 14., 15.]]])


In [14]:
import torch

# 创建一个二维张量
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]]).float()

# 计算每一列的平均值
column_means = torch.mean(tensor, dim=0)

# 将平均值添加为新行
new_row = column_means.unsqueeze(0)  # 添加维度使其成为行向量

# 将新行与原始张量垂直拼接
extended_tensor = torch.cat((tensor, new_row), dim=0)

print(extended_tensor)


tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.],
        [4., 5., 6.]])


In [35]:
import numpy as np

# 假设你的数组名为data
data = np.random.rand(50, 90, 8)  # 这里使用随机生成的数据作为示例

np.array


# 沿着第二个维度计算均值
mean_values = np.mean(data, axis=1)
mean_values = np.mean(mean_values, axis=0)

# 输出结果的形状
print("均值数组的形状:", mean_values.shape)
print(mean_values)

均值数组的形状: (8,)
[0.50591286 0.50035044 0.49669871 0.50322284 0.50480024 0.50018149
 0.50370818 0.50647069]


In [82]:
import torch
import numpy as np

# for i in range(40):
  
x = np.array([[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0]]) # 4 x 3 = (12*1 + 3*10)/15 = 2.8

y = []
y.append(torch.from_numpy(x))
y.append(torch.from_numpy(x))
y.append(torch.from_numpy(x))

y = torch.stack(y) # 等于就是列表转换成矩阵了

# y = torch.tensor(y, dtype=torch.float64)

print(f"{y.shape=}")

# y_mean = torch.mean(y, dim=1)
# y_mean = torch.mean(y_mean, dim=0)
y_mean = y.mean(dim=[0, 1], keepdim=True)
print(f"{y_mean.shape=}")
print(f"{y_mean=}")

# y_std = torch.std(y, dim=1)
# y_std = torch.std(y_std, dim=0)

y_std = y.std(dim=[0, 1], keepdim=True)
print(f"{y_std.shape=}")
print(f"{y_std=}")


y.shape=torch.Size([3, 5, 8])
y_mean.shape=torch.Size([1, 1, 8])
y_mean=tensor([[[ 2.8000,  5.6000,  8.4000, 11.2000, 14.0000, 16.8000, 19.6000,
          22.4000]]], dtype=torch.float64)
y_std.shape=torch.Size([1, 1, 8])
y_std=tensor([[[ 3.7264,  7.4527, 11.1791, 14.9054, 18.6318, 22.3581, 26.0845,
          29.8108]]], dtype=torch.float64)


In [85]:
import torch
import numpy as np

x1 = np.array([[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0]]) # 4 x 3 = (12*1 + 3*10)/15 = 
x2 = np.array([[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0]])

x = np.concatenate((x1, x2), axis=0)


# 不能用append
# 直接拼接

y = []
y.append(x)
x = np.array([[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0]])
y.append(x)
y.append(x)

y = np.array(y)
print(y.shape)

# y = y.reshape(-1,y.shape[-1])
print(y.shape)

y_mean = np.mean(y, axis=(0,1))
print(y_mean)

y_std = np.std(y, axis=(0,1), dtype=np.float64)
print(y_std)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [ ]:
[ 2.8,  5.6,  8.4, 11.2, 14.0, 16.8, 19.6, 22.4]
[ 2.8,  5.6,  8.4, 11.2, 14.0, 16.8, 19.6, 22.4]


[ 3.6  7.2 10.8 14.4 18.  21.6 25.2 28.8]
[ 3.7264,  7.4527, 11.1791, 14.9054, 18.6318, 22.3581, 26.0845, 29.8108]


In [2]:
import torch
import numpy as np
# y = np.array([])

x1 = np.array([[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0]]) # 4 x 3 = (12*1 + 3*10)/15 = 
# x2 = np.array([[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0]])

y = x1
y = np.concatenate((y,x1), axis=0)
y = np.concatenate((y,x1), axis=0)
print(f"{y.shape}")

# y_mean = np.mean(x, axis=0)
# print(y_mean)

# y_std = np.std(x, axis=0, dtype=np.float64)
# print(y_std)

y = torch.from_numpy(y)
print(y.shape)
y_mean = y.mean(dim=0, keepdim=True).unsqueeze(0)
print(f"{y_mean.shape=}")
print(f"{y_mean=}")

y_std = y.std(dim=0, keepdim=True).unsqueeze(0)
print(f"{y_std.shape=}")
print(f"{y_std=}")


(15, 8)
torch.Size([15, 8])
y_mean.shape=torch.Size([1, 1, 8])
y_mean=tensor([[[ 2.8000,  5.6000,  8.4000, 11.2000, 14.0000, 16.8000, 19.6000,
          22.4000]]], dtype=torch.float64)
y_std.shape=torch.Size([1, 1, 8])
y_std=tensor([[[ 3.7264,  7.4527, 11.1791, 14.9054, 18.6318, 22.3581, 26.0845,
          29.8108]]], dtype=torch.float64)


In [5]:
import torch
qpos = torch.tensor([1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0])
mean = torch.tensor([1, 1, 1, 1, 1, 1, 1, 1])
std = 1

print(qpos - std)

qpos_data = torch.tensor([-5.6729e-01, -4.4083e-01,  6.7558e-02,  1.0254e+00, -7.3304e-02,9.9043e-01, -5.3684e-01,  0.0000e+00, -4.8498e-01, -1.0621e-01, 2.7752e-02, -3.4583e-01,  5.7936e-05,  4.5461e-01, -5.3059e-01])

mean = 0.1931866

print(qpos_data - mean)

tensor([0., 1., 2., 3., 4., 5., 6., 7.])
tensor([-0.7605, -0.6340, -0.1256,  0.8322, -0.2665,  0.7972, -0.7300, -0.1932,
        -0.6782, -0.2994, -0.1654, -0.5390, -0.1931,  0.2614, -0.7238])


In [6]:
import numpy as np

# 随机生成 qpos
qpos_shape = (90, 8)
qpos = np.random.rand(*qpos_shape)
qpos_zero = qpos[0]

# 计算差值张量 qpos_diff
for idx in range(qpos.shape[0]):
  

qpos_diff = [a - b for a, b in zip(qpos, qpos_zero)]

# 打印结果
print("qpos 的形状:", qpos.shape)
print("qpos[0] 的形状:", qpos_zero.shape)
print("qpos_diff 的形状:", np.array(qpos_diff).shape)


qpos 的形状: (90, 8)
qpos[0] 的形状: (8,)
qpos_diff 的形状: (8, 8)


In [17]:
import numpy as np

# 随机生成 qpos
qpos_shape = (90, 8)
qpos = np.random.rand(*qpos_shape)
qpos_zero = qpos[0]

# 计算差值张量 qpos_diff
qpos_diff = [a - b for a, b in zip(qpos, qpos_zero)]

# 打印结果
print("qpos 的形状:", qpos.shape)
print("qpos[0] 的形状:", qpos_zero.shape)
print("qpos_diff 的形状:", np.array(qpos_diff).shape)
print(type(qpos_diff))
# 输出代码为什么是
# qpos 的形状: (90, 8)
# qpos[0] 的形状: (8,)
# qpos_diff 的形状: (8, 8)

qpos 的形状: (90, 8)
qpos[0] 的形状: (8,)
qpos_diff 的形状: (8, 8)
<class 'list'>


In [19]:
import numpy as np

# 随机生成 qpos
qpos_shape = (90, 8)
qpos = np.random.rand(*qpos_shape)
print(qpos[7])
qpos_zero = np.array([1,1,1,1,1,1,1,1])

# 计算差值张量 qpos_diff
qpos_diff = np.subtract(qpos, qpos_zero)
# qpos_diff = [a - b for a, b in zip(qpos, qpos_zero)]
print(qpos_diff[7])

# 打印结果
print("qpos 的形状:", qpos.shape)
print("qpos[0] 的形状:", qpos_zero.shape)
print("qpos_diff 的形状:", np.array(qpos_diff).shape)


[0.81076329 0.61987297 0.30150326 0.320093   0.21234729 0.63720904
 0.32008724 0.09214885]
[-0.18923671 -0.38012703 -0.69849674 -0.679907   -0.78765271 -0.36279096
 -0.67991276 -0.90785115]
qpos 的形状: (90, 8)
qpos[0] 的形状: (8,)
qpos_diff 的形状: (90, 8)
